In [54]:
from pyspark import SparkContext
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-518acaae9acb>:2 

In [55]:
import math

In [56]:
# carregar base de dados
import os.path
fileName = os.path.join('C:\spark\Data', 'DelayedFlights.csv')
numPartitions = 2
rawData = sc.textFile(fileName, numPartitions)


In [57]:
Cancelled = rawData.map(lambda x: x.split(",")[22])
featMonth = rawData.map(lambda x: x.split(",")[2])

In [63]:
# carregar base de dados
import os.path
fileName = os.path.join('C:\spark\Data', 'DelayedFlights.csv')

numPartitions = 2
rawData = sc.textFile(fileName, numPartitions)


from collections import Counter    #é o modo certo de fazer utilizando counter?


''''
def Entropia (classe):
    #counts = Counter(classe.collect())  #(tipo: qnt, tipo: qn) #não paralelizado
    
    counts = (classe.map(lambda x: (x, 1))
                   .reduceByKey(lambda a,b: a + b)
             )
    
    n = classe.count()  
    probs = counts.map(lambda x: x[1]/float(n))
    #probs = map( ..., counts.items()) #[c/float(sum(counts)) for _,c in counts]  #não paralelizado
    
    #entropia = -sum([p*math.log(p,2) for p in probs] )  #não paralelizado
    entropia = (probs.map(lambda p: -p*math.log(p,2))
                     .reduce(lambda a,b: a + b))

    return entropia
''''

def Entropia (classe):
     """Calcula a Entropia de Shannon de uma distribuição de dados.

    Args:
        classe (RDD): RDD contendo conjunto de dados a ser calculado a entropia.
                      Valores devem ser categóricos.

    Returns:
        float: (float): valor de Entropia de Shannon calculado para o RDD.
    """
    #counts calcula paralelamente o conteúdo do RDD
    #como tuplas contendo (tipo, quantidade)
    counts = (classe.map(lambda x: (x, 1))             
                    .reduceByKey(lambda a,b: a + b)
             )
    # n recebe o valor total de itens do RDD
    n = classe.count()
    
    # probs calcula a probabilidade de cada um dos estados do RDD
    probs = counts.map(lambda x: x[1]/float(n))
    
    # Entropia calcula a entropia do RDD
    ## a função map faz o calculo da Entropia de cada um dos estados
    ## a função reduce faz o somatório da entropia de Shannon
    entropia = (probs.map(lambda p: -p*math.log(p,2))
                     .reduce(lambda a,b: a + b))
    
    # retorna valor escalar referênte a entropia do RDD.
    return entropia

 
def infoGain (feature, classe, H):
    #feat_count = Counter(feature.collect())  #(tipo: qnt, tipo: qn) #não paralelizado
    feat_count = feature.map(lambda x: (x, 1))\
                        .reduceByKey(lambda a,b: a + b)\
                        .collect()
    print feat_count
    
    feature1 = feature.take(100)
    
    teste = [f==v for v,c in feat_count for f in feature1] 
    print teste
    
    probsCondic = [classe.filter(feature==v).map(Entropia).collect() for v in feat_count]
    
    #probsCondic = [classe.filter(feature==v).map(Entropia).collect() for v in feat_count]

    #probsCondic = [classe.filter(feature==v).map(Entropia).collect() for v,c in feat_count]
    

    
    #probs = probsCondic.map(Entropia)  #como filtrar classe através de um parametro no feature?
    
    
    #print probs
    #probs = [classe.filter(feature==v).map(Entropia) for v,c in feat_count.items()]  #como filtrar classe através de um parametro no feature?

    #print probs
    
    #ig = H - sum([p/float(sum(feat_count))*p for p in probsCondic])
    #print ig
    #return ig
    

In [59]:
H =  Entropia (Cancelled)
print H

0.00425591582633


In [64]:
infoGain(featMonth, Cancelled, H)

[(u'1', 183527), (u'9', 95539), (u'11', 105563), (u'8', 162648), (u'10', 103525), (u'3', 200842), (u'12', 203385), (u'2', 189534), (u'5', 153072), (u'4', 155264), (u'7', 182945), (u'6', 200914)]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, F

In [ ]:
#counts = Counter(classe.collect())  #(tipo: qnt, tipo: qn)
counts = Cancelled.map(lambda x: (x, 1))\
               .reduceByKey(lambda a,b: a + b)
    
print counts.take(2)

In [ ]:

counts = Counter(Cancelled.collect())  #(tipo: qnt, tipo: qn)
print counts

In [ ]:
Cancelled = rawData.map(lambda x: x.split(",")[22])
contagem = Cancelled.map(lambda x: (x, 1))\
                    .reduceByKey(lambda a,b: a + b)
print contagem.take(5)


In [ ]:
Cancelled = rawData.map(lambda x: x.split(",")[22])

featMonth = rawData.map(lambda x: x.split(",")[2])

#H =  Entropia (Cancelled)

In [ ]:
print infoGain (featMonth, Cancelled, H)

In [ ]:
teste = Counter(Cancelled.collect())


In [ ]:
print teste.values()

In [ ]:
sum(teste.values())

In [ ]:
[p*math.log(p) for _,p in test.item]

In [ ]:
print H
